# US Airline Flights Performance Dash Application

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from jupyter_dash import JupyterDash
from dash import no_update
import plotly.graph_objects as go
import plotly.express as px


<ipython-input-1-f3ce1c674e91>:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
<ipython-input-1-f3ce1c674e91>:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Create a dash appliction
app = dash.Dash(__name__)
#JupyterDash.infer_jupyter_proxy_config()


In [3]:
# REVIEW1: Clear the layout and do not display exception till callback gets executed
app.config.suppress_callback_exceptions = True

In [4]:
# Read the airline data into pandas dataframe
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})

In [5]:
# Display from 2005 to 2020
display_years =  [i for i in range(2005, 2021, 1)]

In [6]:
# Function to create graphs for yearly airline performance report
def perfromance_report_data(df):
    # Cancellation Category Count
    bar_data = df.groupby(['Month','CancellationCode'])['Flights'].sum().reset_index()
    # Average flight time by reporting airline
    line_data = df.groupby(['Month','Reporting_Airline'])['AirTime'].mean().reset_index()
    # Diverted Airport Landings
    div_data = df[df['DivAirportLandings'] != 0.0]
    # Source state count
    map_data = df.groupby(['OriginState'])['Flights'].sum().reset_index()
    # Destination state count
    tree_data = df.groupby(['DestState', 'Reporting_Airline'])['Flights'].sum().reset_index()
    return [bar_data, line_data, div_data, map_data, tree_data]

# Function to get yearly flight delay statistics
def avg_flight_delay(df):
    avg_carrier = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_security = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late_aircraft = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return [avg_carrier, avg_weather, avg_NAS, avg_security, avg_late_aircraft]

In [7]:
# Dash Application Layout
app.layout = (html.Div(children=[ 
                                # Add title centre aligned to the dashboard
                                html.H1('US Domestic Airline Flights Performance',
                                        style={'text-align-last':'centre','color':'#503D36','font-size':24}
                                       ),
                                # Dropdown creation
                                # Create an outer division 
                                html.Div([
                                    # Add an division
                                    html.Div([
                                        # Create an division for adding dropdown helper text for report type   
                                        html.Div(
                                            [
                                            html.H2('Report Type:', style={'margin-right': '2em'})
                                            ]
                                        ),
                                        # Add a dropdown
                                        dcc.Dropdown(id='input-type',
                                                     options=[
                                                              {'label': 'Yearly airline performance report', 'value': 'OPT1'},
                                                              {'label': 'Yearly average flight delay statistics', 'value': 'OPT2'}
                                                             ],
                                                     placeholder='Select a report type',
                                                     style={'width':'80%', 'padding':'3px', 'font-size':'20px', 'text-align-last':'center'}
                                                    )
                                    # Place them next to each other using the division style
                                            ], style={'display':'flex'}),
                                    
                                   # Add next division 
                                   html.Div([
                                       # Create an division for adding dropdown helper text for choosing year
                                        html.Div(
                                            [
                                            html.H2('Choose Year:', style={'margin-right': '2em'})
                                            ]
                                        ),
                                        dcc.Dropdown(id='input-year', 
                                                     # Update dropdown values using list comphrehension
                                                     options=[{'label': i, 'value': i} for i in display_years],
                                                     placeholder="Select a year",
                                                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                            # Place them next to each other using the division style
                                            ], style={'display': 'flex'})  
                                          ]),
                                
                                # Add Computed graphs
                                # Observe how we add an empty division and providing an id that will be updated during callback
                                html.Div([ ], id='plot1'),
    
                                html.Div([
                                          html.Div([ ], id='plot2'),
                                          html.Div([ ], id='plot3')
                                         ], 
                                          style={'display': 'flex'}),
                                
                                # Add a division with two empty divisions inside. See above disvision for example.
                                html.Div([
                                          html.Div([ ], id='plot4'),
                                          html.Div([ ], id='plot5')
                                         ], 
                                          style={'display': 'flex'})
                                ])
             )

In [8]:
# Callback function definition
# Add 5 ouput components
@app.callback( [Output(component_id='plot1', component_property='children'),
                Output(component_id='plot2', component_property='children'),
                Output(component_id='plot3', component_property='children'),
                Output(component_id='plot4', component_property='children'),
                Output(component_id='plot5', component_property='children')
               ],
               [Input(component_id='input-type', component_property='value'),
                Input(component_id='input-year', component_property='value')
               ],
               # Holding output state till user enters all the form information. In this case, it will be chart type and year
               [State("plot1", 'children'), 
                State("plot2", "children"),
                State("plot3", "children"), 
                State("plot4", "children"),
                State("plot5", "children")
               ])

# Add computation to callback function and return graph
def get_graph(chart, year, children1, children2, c3, c4, c5):
      
        # Select data
        df =  airline_data[airline_data['Year']==int(year)]
       
        if chart == 'OPT1':
            # Compute required information for creating graph from the data
            bar_data, line_data, div_data, map_data, tree_data = perfromance_report_data(df)
            
            # Number of flights under different cancellation categories
            bar_fig = px.bar(bar_data, x='Month', y='Flights', color='CancellationCode', title='Monthly Flight Cancellation')
            
            # Average flight time by reporting airline
            line_fig = px.line(line_data, x='Month', y='AirTime', color='Reporting_Airline', title='Average monthly flight time (minutes) by airline')
            
            # Percentage of diverted airport landings per reporting airline
            pie_fig = px.pie(div_data, values='Flights', names='Reporting_Airline', title='% of flights by reporting airline')
            
            # Number of flights flying from each state using choropleth
            map_fig = px.choropleth(map_data,  # Input data
                    locations='OriginState', 
                    color='Flights',  
                    hover_data=['OriginState', 'Flights'], 
                    locationmode = 'USA-states', # Set to plot as USA States
                    color_continuous_scale='GnBu',
                    range_color=[0, map_data['Flights'].max()]) 
            map_fig.update_layout(
                    title_text = 'Number of flights from origin state', 
                    geo_scope='usa') # Plot only the USA instead of globe
            
            # Number of flights flying to each state from each reporting airline
            tree_fig = px.treemap(tree_data, path=['DestState', 'Reporting_Airline'], 
                      values='Flights',
                      color='Flights',
                      color_continuous_scale='RdBu',
                      title='Flight count by airline to destination state'
                )
            
            # Return dcc.Graph component to the empty division
            return [dcc.Graph(figure=tree_fig), 
                    dcc.Graph(figure=pie_fig),
                    dcc.Graph(figure=map_fig),
                    dcc.Graph(figure=bar_fig),
                    dcc.Graph(figure=line_fig)
                   ]
        else:
           
            # Compute required information for creating graph from the data
            avg_carrier, avg_weather, avg_NAS, avg_security, avg_late_aircraft = avg_flight_delay(df)
            
            # Create graph
            carrier_fig = px.line(avg_carrier, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
            weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
            NAS_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
            security_fig = px.line(avg_security, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
            late_aircraft_fig = px.line(avg_late_aircraft, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
            
            return[dcc.Graph(figure=carrier_fig), 
                   dcc.Graph(figure=weather_fig), 
                   dcc.Graph(figure=NAS_fig), 
                   dcc.Graph(figure=security_fig), 
                   dcc.Graph(figure=late_aircraft_fig)]

In [ ]:
# Run dash app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Jan/2022 09:13:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:13:49] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:13:49] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:13:49] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Owner\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [06/Jan/2022 09:13:49] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Owner\anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\Owner\anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_d

127.0.0.1 - - [06/Jan/2022 09:13:51] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [06/Jan/2022 09:13:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:13:53] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:13:53] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:16:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:16:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:20:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:21:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:21:49] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:22:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [06/Jan/2022 09:26:15] "